In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim 
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from torch.utils.data import DataLoader,Dataset
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.decomposition import PCA

/home/sjlee/miniconda3/envs/sj_virtual/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [4]:
# 절반 이상이 NaN인 데이터 column 드롭
drop_col = train.loc[:,(train.isna().sum(axis=0)>=len(train)/2).values].columns
train.drop(columns = drop_col,inplace = True)
train

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [5]:
# 전부 다 다른 column과 전부 다 같은 column도 드롭
print(len(train.columns[(train.nunique()==1)|(train.nunique()==len(train))]))
train.drop(columns=train.columns[(train.nunique()==1)|(train.nunique()==len(train))],inplace=True)

35


In [6]:
fill_col = train.columns[train.isna().sum(axis=0)>0]
fill_col

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [7]:
no_agreement_col_list = train.columns[(train.applymap(lambda x: type(x).__name__).nunique()>1).values]
no_agreement_col_list

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [8]:
# 불일치 컬럼의 dtype 조사
# OK 부분이 잘못됐음
condition = (train.loc[:,no_agreement_col_list] != 'OK')

In [9]:
replace_list = []
replace_list.append(train.loc[condition.iloc[:,0],no_agreement_col_list[0]].dropna().astype(float).mean())
replace_list.append(train.loc[condition.iloc[:,1],no_agreement_col_list[1]].dropna().astype(float).mean())
replace_list.append(train.loc[condition.iloc[:,2],no_agreement_col_list[2]].dropna().astype(float).mean())

In [10]:
print(replace_list)

[465.18351067063895, 838.0938773028515, 720.05891651973]


In [11]:
train.loc[:,no_agreement_col_list[0]] = train.loc[:,no_agreement_col_list[0]].replace({'OK':replace_list[0]})
train.loc[:,no_agreement_col_list[1]] = train.loc[:,no_agreement_col_list[1]].replace({'OK':replace_list[1]})
train.loc[:,no_agreement_col_list[2]] = train.loc[:,no_agreement_col_list[2]].replace({'OK':replace_list[2]})

In [12]:
# 결측치 평균으로 제거
train.loc[:,no_agreement_col_list[0]].fillna(replace_list[0],inplace=True)
train.loc[:,no_agreement_col_list[1]].fillna(replace_list[1],inplace=True)
train.loc[:,no_agreement_col_list[2]].fillna(replace_list[2],inplace=True)

In [13]:
train[no_agreement_col_list[0]] = train.loc[:,no_agreement_col_list[0]].astype(float)
train[no_agreement_col_list[1]] = train.loc[:,no_agreement_col_list[1]].astype(float)
train[no_agreement_col_list[2]] = train.loc[:,no_agreement_col_list[2]].astype(float)

In [14]:
print(train.isna().sum().sum())

0


In [15]:
train.reset_index(inplace=True,drop=True)

In [16]:
train.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

In [17]:
# 수치 data만 뽑음
condition = ((train.dtypes==float) | (train.dtypes==int)).values
numeric_column = train.columns[condition].values
# 3,10,14 index의 object column이 float인데 object로 표시됨
object_column = train.columns[~condition].values
#train.loc[:,object_column[]]
print(object_column)

['Equipment_Dam' 'Model.Suffix_Dam' 'Workorder_Dam'
 'Model.Suffix_AutoClave' 'Workorder_AutoClave'
 'Chamber Temp. Judge Value_AutoClave' 'Equipment_Fill1'
 'Model.Suffix_Fill1' 'Workorder_Fill1' 'Equipment_Fill2'
 'Model.Suffix_Fill2' 'Workorder_Fill2' 'target']


In [18]:
normal_index = train[train['target']=='Normal'].index
abnormal_index = train[train['target']=='AbNormal'].index
print(normal_index,abnormal_index)

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       40494, 40495, 40497, 40498, 40499, 40500, 40501, 40502, 40503, 40504],
      dtype='int64', length=38156) Index([   17,    38,    44,    59,    83,   126,   137,   177,   193,   227,
       ...
       40368, 40372, 40374, 40395, 40405, 40437, 40463, 40481, 40496, 40505],
      dtype='int64', length=2350)


In [19]:
reduction_train = train.copy()
reduction_train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [20]:
pca = PCA(n_components=24)
concat_item =\
      pca.fit_transform(reduction_train.loc[:,numeric_column])

In [21]:
pca.components_.shape

(24, 133)

In [22]:
reduction_column = [f'PC{i}' for i in range(1,25)]

In [23]:
print(concat_item.shape)
concat_df = pd.DataFrame(concat_item,columns = reduction_column)
concat_df

(40506, 24)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
0,-6450.270542,-1892.668992,-124.605150,-1497.902988,-318.247446,169.136518,140.663222,-24.780679,-75.422349,-20.606696,...,-36.165382,9.029621,0.536252,50.468853,-11.199051,11.263131,15.830669,-8.525877,-6.338858,-4.916051
1,-1119.381735,11120.002183,2082.920657,-1379.108455,-584.297003,-823.943684,-123.001025,-67.541019,-27.731979,55.329281,...,0.746238,0.897786,-0.230213,-1.936211,0.764078,11.443625,-8.552788,3.026316,-2.250788,-4.974201
2,-6603.526427,-1978.620294,242.679268,1505.508315,-166.537139,-327.599213,119.325224,-6.879560,-169.570296,-14.108013,...,2.093321,12.878064,23.532843,11.948065,-5.272283,-25.757930,-26.744306,-2.456295,14.298813,22.573445
3,-8532.417498,6554.246788,-4877.953026,1576.619574,-121.202603,1185.904803,-50.075415,7.375831,-55.991571,248.063137,...,-2.464069,-2.138953,1.021040,4.138621,-1.400113,-40.648913,34.955521,-10.702608,6.718841,18.469064
4,12761.271276,-2190.598837,-3088.956112,2.099439,336.212055,-25.445304,-5.610929,-11.902722,-4.736689,-43.067797,...,0.190174,-0.087716,-0.126562,-0.594188,0.411697,7.081970,-5.627452,1.049559,-1.097017,0.340618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,1659.727879,8463.142549,1044.623869,-1100.822316,-406.278111,-660.592216,-104.748473,-82.760827,-29.493817,276.376746,...,0.400060,0.100812,-0.487405,-1.872040,0.960580,12.866215,-10.595693,4.419902,-2.515773,-9.824598
40502,-6638.993703,-1944.547164,256.035007,1511.468790,-156.881162,-332.800262,103.914234,-15.782497,37.763879,-1.715351,...,-47.155977,6.713139,22.567960,61.768574,-14.946250,-7.219100,-1.844990,-0.372004,-1.144330,-10.347392
40503,-5616.403493,-2360.299306,227.073535,-1496.520152,-330.327351,208.155838,168.756305,-14.028102,-248.819368,-24.916803,...,14.211626,15.234624,2.266028,-4.678669,-1.047513,-5.579171,-6.916523,-7.925458,8.447319,19.269575
40504,12641.699918,-2060.959535,-2847.536378,2552.870919,106.588634,1579.013674,134.498539,43.743791,31.025255,-122.530755,...,-0.109037,-0.187916,-2.435546,0.659677,1.096057,9.113804,-9.339019,4.363699,-1.542500,-13.838144


In [24]:
object_column

array(['Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam',
       'Model.Suffix_AutoClave', 'Workorder_AutoClave',
       'Chamber Temp. Judge Value_AutoClave', 'Equipment_Fill1',
       'Model.Suffix_Fill1', 'Workorder_Fill1', 'Equipment_Fill2',
       'Model.Suffix_Fill2', 'Workorder_Fill2', 'target'], dtype=object)

In [25]:
reduction_train = pd.concat([reduction_train.loc[:,object_column],concat_df],axis=1,ignore_index=True)
reduction_train

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,Dam dispenser #1,AJX75334505,4F1XA938-1,AJX75334505,4F1XA938-1,OK,Fill1 dispenser #1,AJX75334505,4F1XA938-1,Fill2 dispenser #1,...,-36.165382,9.029621,0.536252,50.468853,-11.199051,11.263131,15.830669,-8.525877,-6.338858,-4.916051
1,Dam dispenser #1,AJX75334505,3KPM0016-2,AJX75334505,3KPM0016-2,OK,Fill1 dispenser #1,AJX75334505,3KPM0016-2,Fill2 dispenser #1,...,0.746238,0.897786,-0.230213,-1.936211,0.764078,11.443625,-8.552788,3.026316,-2.250788,-4.974201
2,Dam dispenser #2,AJX75334501,4E1X9167-1,AJX75334501,4E1X9167-1,OK,Fill1 dispenser #2,AJX75334501,4E1X9167-1,Fill2 dispenser #2,...,2.093321,12.878064,23.532843,11.948065,-5.272283,-25.757930,-26.744306,-2.456295,14.298813,22.573445
3,Dam dispenser #2,AJX75334501,3K1X0057-1,AJX75334501,3K1X0057-1,OK,Fill1 dispenser #2,AJX75334501,3K1X0057-1,Fill2 dispenser #2,...,-2.464069,-2.138953,1.021040,4.138621,-1.400113,-40.648913,34.955521,-10.702608,6.718841,18.469064
4,Dam dispenser #1,AJX75334501,3HPM0007-1,AJX75334501,3HPM0007-1,NG,Fill1 dispenser #1,AJX75334501,3HPM0007-1,Fill2 dispenser #1,...,0.190174,-0.087716,-0.126562,-0.594188,0.411697,7.081970,-5.627452,1.049559,-1.097017,0.340618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,AJX75334501,3J1XF434-2,OK,Fill1 dispenser #1,AJX75334501,3J1XF434-2,Fill2 dispenser #1,...,0.400060,0.100812,-0.487405,-1.872040,0.960580,12.866215,-10.595693,4.419902,-2.515773,-9.824598
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,AJX75334501,4E1XC796-1,NG,Fill1 dispenser #2,AJX75334501,4E1XC796-1,Fill2 dispenser #2,...,-47.155977,6.713139,22.567960,61.768574,-14.946250,-7.219100,-1.844990,-0.372004,-1.144330,-10.347392
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,AJX75334501,4C1XD438-1,NG,Fill1 dispenser #1,AJX75334501,4C1XD438-1,Fill2 dispenser #1,...,14.211626,15.234624,2.266028,-4.678669,-1.047513,-5.579171,-6.916523,-7.925458,8.447319,19.269575
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,AJX75334501,3I1XA258-1,OK,Fill1 dispenser #2,AJX75334501,3I1XA258-1,Fill2 dispenser #2,...,-0.109037,-0.187916,-2.435546,0.659677,1.096057,9.113804,-9.339019,4.363699,-1.542500,-13.838144


In [26]:
reduction_train.to_csv('pca_train.csv',index=False)

# Test Set 성능 측정

In [27]:
test = pd.read_csv('./data/test.csv')
test = test[train.columns]
test

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.8,13.0,195,1,0,NaN
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,19.8,14,256,1,NaN
2,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.7,1.0,98,1,0,NaN
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,280,90,10,...,50.0,91.8,270,50,85.0,20.0,14,0,1,NaN
4,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1030,-90,16,...,50.0,91.8,270,50,85.0,19.8,1,215,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,280,90,10,...,50.0,91.8,270,50,85.0,19.5,14,131,1,NaN
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,19.8,12,279,1,NaN
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1030,-90,16,...,50.0,91.8,270,50,85.0,20.5,4,66,1,NaN
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,18.9,1.0,117,1,0,NaN


In [28]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [29]:
no_agreement_col_list = test.columns[(test.applymap(lambda x: type(x).__name__).nunique()>1).values]
no_agreement_col_list

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [30]:
condition = ~(test.loc[:,no_agreement_col_list].isin(['OK','NG']))
condition

,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2
0,True,True,True
1,False,False,False
2,True,True,True
3,True,True,True
4,False,False,False
...,...,...,...
17356,True,True,True
17357,False,False,False
17358,True,True,True
17359,True,True,True


In [31]:
replace_list = []
replace_list.append(test.loc[condition.iloc[:,0],no_agreement_col_list[0]].dropna().astype(float).mean())
replace_list.append(test.loc[condition.iloc[:,1],no_agreement_col_list[1]].dropna().astype(float).mean())
replace_list.append(test.loc[condition.iloc[:,2],no_agreement_col_list[2]].dropna().astype(float).mean())

In [32]:
test.loc[:,no_agreement_col_list[0]] = test.loc[:,no_agreement_col_list[0]].replace({'OK':replace_list[0]})
test.loc[:,no_agreement_col_list[1]] = test.loc[:,no_agreement_col_list[1]].replace({'OK':replace_list[1]})
test.loc[:,no_agreement_col_list[2]] = test.loc[:,no_agreement_col_list[2]].replace({'OK':replace_list[2]})

In [33]:
# 결측치 평균으로 제거
test.loc[:,no_agreement_col_list[0]].fillna(replace_list[0],inplace=True)
test.loc[:,no_agreement_col_list[1]].fillna(replace_list[1],inplace=True)
test.loc[:,no_agreement_col_list[2]].fillna(replace_list[2],inplace=True)

In [34]:
test[no_agreement_col_list[0]] = test.loc[:,no_agreement_col_list[0]].astype(float)
test[no_agreement_col_list[1]] = test.loc[:,no_agreement_col_list[1]].astype(float)
test[no_agreement_col_list[2]] = test.loc[:,no_agreement_col_list[2]].astype(float)

In [37]:
(test.dtypes==object).sum()

12

In [38]:
reduction_test = test.copy()

In [39]:
concat_item =\
      pca.transform(reduction_test.loc[:,numeric_column])

In [40]:
reduction_column = [f'PC{i}' for i in range(1,25)]

In [41]:
print(concat_item.shape)
concat_df = pd.DataFrame(concat_item,columns = reduction_column)
concat_df

(17361, 24)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
0,1537.304097,8587.737642,1290.009183,1450.225097,-629.909367,939.478518,40.401104,-4.461692,12.247941,-2.617877,...,0.441822,0.455273,0.269325,-1.238152,0.492180,6.959485,-5.926402,2.009640,-1.824390,-2.626029
1,-5808.053549,-2417.458156,613.402481,1507.128668,-163.966979,-292.776271,105.365587,29.980527,134.192719,21.133866,...,4.821423,9.639068,20.259314,3.928456,-4.542729,-22.891991,-21.994288,0.685102,10.901917,14.508316
2,12760.987225,-2190.877594,-3088.680714,2.062191,336.502649,-24.704172,-5.284681,-8.450458,-4.242042,-78.890534,...,0.448806,-0.000310,-0.845370,-2.121011,1.438319,12.259931,-8.430807,4.413903,-2.896132,-13.369294
3,-12905.684087,2384.216791,-2237.460617,1387.508347,453.210731,-565.742363,20.817770,2.694342,-234.978249,-89.968387,...,6.023040,7.138623,-20.968944,-10.638637,5.445040,19.410360,18.703112,-12.371858,2.363786,23.805340
4,-3197.746113,-3308.233298,1520.130087,-1515.526584,157.065093,468.166861,-989.469001,-15.239315,-61.693500,-21.868368,...,4.393796,1.432299,-10.076963,-4.993634,1.617880,9.122725,8.559979,-4.423807,-1.227006,2.612745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,-12908.366692,2379.953096,-2232.846819,1386.334529,458.793269,-567.652557,-0.906401,33.374066,-11.602079,-78.024391,...,8.526524,4.206943,-23.325635,-22.888143,6.355010,23.373249,23.745441,-1.180442,-9.862477,-8.717693
17357,-3385.885869,-3359.517113,1899.730815,1489.576267,317.639263,-32.351600,-1025.448912,-8.412412,41.092907,3.542244,...,-6.189665,-0.705675,9.064880,5.279408,-1.777022,-8.853498,-7.986885,4.092596,1.211678,-2.399325
17358,-2732.416595,-2977.588266,1937.937554,-1544.751457,248.721655,375.276706,243.146506,-27.712176,-207.404441,17.374441,...,15.151955,11.462799,-2.212937,-18.890753,2.500339,-0.450027,2.360564,0.850209,-1.942946,-4.352603
17359,18232.107906,1331.921148,2312.770426,-299.326350,-185.230291,-330.145640,43.809360,-36.242310,41.812710,-151.588319,...,2.148078,2.285775,0.859113,-2.722195,-0.715220,-22.827568,21.756469,-1.748279,-0.944664,-2.326973


In [42]:
object_column

array(['Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam',
       'Model.Suffix_AutoClave', 'Workorder_AutoClave',
       'Chamber Temp. Judge Value_AutoClave', 'Equipment_Fill1',
       'Model.Suffix_Fill1', 'Workorder_Fill1', 'Equipment_Fill2',
       'Model.Suffix_Fill2', 'Workorder_Fill2', 'target'], dtype=object)

In [43]:
reduction_test = pd.concat([reduction_test.loc[:,object_column],concat_df],axis=1,ignore_index=True)
reduction_test

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,Dam dispenser #2,AJX75334501,3J1XF767-1,AJX75334501,3J1XF767-1,OK,Fill1 dispenser #2,AJX75334501,3J1XF767-1,Fill2 dispenser #2,...,0.441822,0.455273,0.269325,-1.238152,0.492180,6.959485,-5.926402,2.009640,-1.824390,-2.626029
1,Dam dispenser #2,AJX75334501,4B1XD472-2,AJX75334501,4B1XD472-2,OK,Fill1 dispenser #2,AJX75334501,4B1XD472-2,Fill2 dispenser #2,...,4.821423,9.639068,20.259314,3.928456,-4.542729,-22.891991,-21.994288,0.685102,10.901917,14.508316
2,Dam dispenser #1,AJX75334501,3H1XE355-1,AJX75334501,3H1XE355-1,OK,Fill1 dispenser #1,AJX75334501,3H1XE355-1,Fill2 dispenser #1,...,0.448806,-0.000310,-0.845370,-2.121011,1.438319,12.259931,-8.430807,4.413903,-2.896132,-13.369294
3,Dam dispenser #2,AJX75334501,3L1XA128-1,AJX75334501,3L1XA128-1,OK,Fill1 dispenser #2,AJX75334501,3L1XA128-1,Fill2 dispenser #2,...,6.023040,7.138623,-20.968944,-10.638637,5.445040,19.410360,18.703112,-12.371858,2.363786,23.805340
4,Dam dispenser #1,AJX75334501,4A1XA639-1,AJX75334501,4A1XA639-1,OK,Fill1 dispenser #1,AJX75334501,4A1XA639-1,Fill2 dispenser #1,...,4.393796,1.432299,-10.076963,-4.993634,1.617880,9.122725,8.559979,-4.423807,-1.227006,2.612745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,AJX75334501,3K1XB597-1,OK,Fill1 dispenser #2,AJX75334501,3K1XB597-1,Fill2 dispenser #2,...,8.526524,4.206943,-23.325635,-22.888143,6.355010,23.373249,23.745441,-1.180442,-9.862477,-8.717693
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,AJX75334501,4A1XB974-1,OK,Fill1 dispenser #2,AJX75334501,4A1XB974-1,Fill2 dispenser #2,...,-6.189665,-0.705675,9.064880,5.279408,-1.777022,-8.853498,-7.986885,4.092596,1.211678,-2.399325
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,AJX75334501,3L1XA998-1,OK,Fill1 dispenser #1,AJX75334501,3L1XA998-1,Fill2 dispenser #1,...,15.151955,11.462799,-2.212937,-18.890753,2.500339,-0.450027,2.360564,0.850209,-1.942946,-4.352603
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,AJX75334501,3F1XC376-1,OK,Fill1 dispenser #1,AJX75334501,3F1XC376-1,Fill2 dispenser #1,...,2.148078,2.285775,0.859113,-2.722195,-0.715220,-22.827568,21.756469,-1.748279,-0.944664,-2.326973


In [44]:
reduction_test.to_csv('pca_test.csv',index=False)